In [1]:
!pip install ace-tools

Defaulting to user installation because normal site-packages is not writeable


In [9]:
df["weather"].value_counts()

,count
weather,
1,7192
2,2834
3,859
4,1


In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import joblib


df = pd.read_csv('/content/train.csv', parse_dates=['datetime'])

# Extract hour
df['hour'] = df['datetime'].dt.hour

# Create deterministic binary "event" feature:
df['event'] = (
    df['datetime'].dt.month.isin([6, 7, 8]) &
    df['hour'].between(8, 20) &
    (df['weather'] == 1) &
    (df['workingday'] == 1)
).astype(int)

# Define features and target
features = ['season', 'holiday', 'workingday', 'weather',
            'temp', 'atemp', 'humidity', 'windspeed', 'event']
X = df[features]
y = df['count']

# Train linear regression model
model = LinearRegression()
model.fit(X, y)

# Save the trained model to the same directory
model_path = '/content/linear_regression_model.pkl'
joblib.dump(model, model_path)

# Prepare coefficient DataFrame for interpretability
coef_df = pd.DataFrame({
    'feature': features,
    'coefficient': model.coef_
})

# Display the coefficients to the user
display('Feature Coefficients', coef_df)

print(f"Model retrained and saved to: {model_path}")

'Feature Coefficients'

,feature,coefficient
0,season,22.949566
1,holiday,-8.811922
2,workingday,-4.279345
3,weather,7.431437
4,temp,0.715318
5,atemp,6.304548
6,humidity,-2.999312
7,windspeed,0.807972
8,event,32.919399


Model retrained and saved to: /content/linear_regression_model.pkl


In [12]:
from flask import Flask, request, jsonify, render_template
import joblib
import pandas as pd
import os

# Initialize Flask app
app = Flask(__name__)

# Load the trained linear regression model
MODEL_PATH = '/content/linear_regression_model.pkl'
model = joblib.load(MODEL_PATH)

# Define expected features
FEATURES = ['season', 'holiday', 'workingday', 'weather',
            'temp', 'atemp', 'humidity', 'windspeed', 'event']

@app.route('/')
def home():
    """
    Render a simple HTML form for manual input.
    """
    return render_template('index.html', features=FEATURES)

@app.route('/predict', methods=['POST'])
def predict():
    """
    Accept JSON payload or form data to predict bike demand.
    JSON format:
    {
      "season": 1,
      "holiday": 0,
      ...
    }
    """
    # Determine data source
    data = request.get_json(silent=True)
    if data is None:
        # Fallback to form data
        data = {feat: request.form.get(feat, type=float) for feat in FEATURES}
    # Create DataFrame from input
    input_df = pd.DataFrame([data], columns=FEATURES)
    # Generate prediction
    pred = model.predict(input_df)[0]
    # Return as JSON
    return jsonify({"predicted_count": float(pred)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
